In [1]:
# Import libraries
import pandas as pd
import numpy as np
import ast
import re
import math
from geopy.geocoders import Nominatim
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from folium.plugins import FloatImage
import folium # plotting library
from folium.plugins import HeatMap

import matplotlib.pyplot as plt
import seaborn as sns

# Plotly
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import chart_studio.plotly as py
init_notebook_mode(connected=True)
%matplotlib inline


In [2]:
rest_df = pd.read_csv(r"C:\Users\Dinesh\Coursera_Capstone-master\Final Project\rest\zomato.csv", encoding='utf-8')


In [3]:
locations=pd.DataFrame({"Name":rest_df['location'].unique()})
locations['Name']=locations['Name'].apply(lambda x: "Bangalore " + str(x))
lat_lon=[]
geolocator=Nominatim(user_agent="app")
for location in locations['Name']:
    location = geolocator.geocode(location, timeout= 20)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)


locations['geo_loc']=lat_lon
locations.to_csv('locations.csv',index=False)

In [4]:
locations["Name"]=locations['Name'].apply(lambda x :  x.replace("Bangalore","")[1:])
locations.head()

,Name,geo_loc
0,Banashankari,"(12.9152208, 77.573598)"
1,Basavanagudi,"(12.9417261, 77.5755021)"
2,Mysore Road,"(12.5523878, 76.9685867)"
3,Jayanagar,"(12.9292731, 77.5824229)"
4,Kumaraswamy Layout,"(12.906816899999999, 77.56352473621337)"


In [5]:
rest_count= pd.DataFrame(rest_df['location'].value_counts().reset_index())
rest_count.columns= ['Name', 'count']
rest_count=rest_count.merge(locations, on = "Name", how = "left" ).dropna()
rest_count.head()

,Name,count,geo_loc
0,BTM,5124,"(12.911275849999999, 77.60456543431182)"
1,HSR,2523,"(12.91445305, 77.64269383637028)"
2,Koramangala 5th Block,2504,"(13.2923988, 77.7519261)"
3,JP Nagar,2235,"(12.9072515, 77.5782713)"
4,Whitefield,2144,"(12.9696365, 77.7497448)"


In [6]:
maps = folium.Map(location= [12.97 , 77.59], zoom_start= 12)
folium.Marker(
location= [12.97 , 77.59], popup= 'geographical center of Bangaluru', 
    icon= folium.Icon(color= 'green', icon= 'ok-sign')).add_to(maps)
lat , lon= zip(*np.array(rest_count['geo_loc']))
rest_count['lat']= lat
rest_count['lon']= lon

HeatMap(rest_count[['lat', 'lon', 'count']].values.tolist()).add_to(maps)
maps
##We Observe That:

#Restaurants are concentrated towards the center of Bangaluru.
#Central Bangaluru is the best place for starting new restautrants.



In [7]:
#FINDING NORTH INDIAN RESTAURANTS IN BANGALURU


north_data=  pd.DataFrame(rest_df[rest_df['cuisines']== 'North Indian'].groupby(['location'], as_index= False)['url'].agg('count'))
north_data.columns= ['Name', 'count']
#north_data.head()
north_data = north_data.merge(locations, on = 'Name', how = 'left' ).dropna()
#north_data.head()
lat , lon= zip(*np.array(north_data['geo_loc']))
north_data['lat'] = lat
north_data['lon'] = lon
north_data.head()

,Name,count,geo_loc,lat,lon
0,BTM,340,"(12.911275849999999, 77.60456543431182)",12.911276,77.604565
1,Banashankari,36,"(12.9152208, 77.573598)",12.915221,77.573598
2,Banaswadi,12,"(13.0141618, 77.6518539)",13.014162,77.651854
3,Bannerghatta Road,75,"(12.8264929, 77.5864733)",12.826493,77.586473
4,Basavanagudi,21,"(12.9417261, 77.5755021)",12.941726,77.575502


In [8]:
maps= folium.Map(location = [12.97 , 77.59], zoom_start= 12)
folium.Marker(
location= [12.97 , 77.59], popup= 'geographical center of Bangaluru', 
    icon= folium.Icon(color= 'green', icon= 'ok-sign')).add_to(maps)
HeatMap(north_data[['lat', 'lon', 'count']].values.tolist()).add_to(maps)
maps

###WE OBSERVE THAT:

###1. North Indian Restaurants are more popular in South Bangaluru region.


In [9]:
#FINDING SOUTH INDIAN RESTAURANTS IN BANGALURU


south_data=  pd.DataFrame(rest_df[rest_df['cuisines']== 'South Indian'].groupby(['location'], as_index= False)['url'].agg('count'))
south_data.columns= ['Name', 'count']
south_data = south_data.merge(locations, on = 'Name', how = 'left' ).dropna()
lat , lon= zip(*np.array(south_data['geo_loc']))
south_data['lat'] = lat
south_data['lon'] = lon
south_data.head()

,Name,count,geo_loc,lat,lon
0,BTM,113,"(12.911275849999999, 77.60456543431182)",12.911276,77.604565
1,Banashankari,86,"(12.9152208, 77.573598)",12.915221,77.573598
2,Banaswadi,37,"(13.0141618, 77.6518539)",13.014162,77.651854
3,Bannerghatta Road,48,"(12.8264929, 77.5864733)",12.826493,77.586473
4,Basavanagudi,89,"(12.9417261, 77.5755021)",12.941726,77.575502


In [10]:
maps= folium.Map(location = [12.97 , 77.59], zoom_start= 12)
folium.Marker(
location= [12.97 , 77.59], popup= 'geographical center of Bangaluru', 
    icon= folium.Icon(color= 'green', icon= 'ok-sign')).add_to(maps)
HeatMap(south_data[['lat', 'lon', 'count']].values.tolist()).add_to(maps)
maps

In [16]:
rest_df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [11]:
rest_df['location'].unique()

array(['Banashankari', 'Basavanagudi', 'Mysore Road', 'Jayanagar',
       'Kumaraswamy Layout', 'Rajarajeshwari Nagar', 'Vijay Nagar',
       'Uttarahalli', 'JP Nagar', 'South Bangalore', 'City Market',
       'Nagarbhavi', 'Bannerghatta Road', 'BTM', 'Kanakapura Road',
       'Bommanahalli', nan, 'CV Raman Nagar', 'Electronic City', 'HSR',
       'Marathahalli', 'Sarjapur Road', 'Wilson Garden', 'Shanti Nagar',
       'Koramangala 5th Block', 'Koramangala 8th Block', 'Richmond Road',
       'Koramangala 7th Block', 'Jalahalli', 'Koramangala 4th Block',
       'Bellandur', 'Whitefield', 'East Bangalore', 'Old Airport Road',
       'Indiranagar', 'Koramangala 1st Block', 'Frazer Town', 'RT Nagar',
       'MG Road', 'Brigade Road', 'Lavelle Road', 'Church Street',
       'Ulsoor', 'Residency Road', 'Shivajinagar', 'Infantry Road',
       'St. Marks Road', 'Cunningham Road', 'Race Course Road',
       'Commercial Street', 'Vasanth Nagar', 'HBR Layout', 'Domlur',
       'Ejipura', 'Jeevan 

In [12]:
rest_df['location']=rest_df.location.str.replace('Koramangala 1st Block,?' , '')
rest_df['location']=rest_df.location.str.replace('Rammurthy Nagar,?' , '')
rest_df['location']=rest_df.location.str.replace('Sankey Road,?' , '')

In [13]:
location = [x for x in rest_df['location'].unique().tolist() if type(x) == str]
latitude = []
longitude =  []
for i in range(0, len(location)):
    if(type(location[i]) == str):
        ctr=0
        while True:
            try:
                address = location[i] + ', Bengaluru, India'
                geolocator = Nominatim(user_agent="ny_explorer")
                loc = geolocator.geocode(address)
                latitude.append(loc.latitude)
                longitude.append(loc.longitude)
                print('The geograpical coordinate of location are {}, {}.'.format(loc.latitude, loc.longitude))
            except:
                ctr+=1
                if(ctr==7):
                    print(i)
                    latitude.append(address)
                    longitude.append(address)
                    break
                continue
            break

The geograpical coordinate of location are 12.9152208, 77.573598.
The geograpical coordinate of location are 12.9417261, 77.5755021.
The geograpical coordinate of location are 12.9510141, 77.5387513.
The geograpical coordinate of location are 12.9292731, 77.5824229.
The geograpical coordinate of location are 12.906816899999999, 77.56352473621337.
The geograpical coordinate of location are 12.9274413, 77.5155224.
The geograpical coordinate of location are 12.9662966, 77.61272241867627.
The geograpical coordinate of location are 12.9055682, 77.5455438.
The geograpical coordinate of location are 12.9072515, 77.5782713.
The geograpical coordinate of location are 13.0646907, 77.49626895712257.
The geograpical coordinate of location are 12.965717999999999, 77.5762705372058.
The geograpical coordinate of location are 12.9546741, 77.5121724.
The geograpical coordinate of location are 12.8264929, 77.5864733.
The geograpical coordinate of location are 12.911275849999999, 77.60456543431182.
The g

In [14]:
rest_df['location_latitude'] = rest_df['location'].map(dict(zip(location, latitude)))
rest_df['location_longitude'] = rest_df['location'].map(dict(zip(location, longitude)))

In [15]:
dataframe_filtered = rest_df.groupby(['location'])['location_latitude', 'location_longitude'].first()
dataframe_filtered['no_restaurant'] = rest_df.groupby(['location'])['url'].count()
venues_map = folium.Map(location=[12.934533, 77.626579], zoom_start=11) # generate map centred around Bengaluru

states = folium.map.FeatureGroup()
i=0
for lat, lng, in zip(dataframe_filtered.location_latitude, dataframe_filtered.location_longitude):
    states.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
        )
    )
    i+=1
i=0
for lat, lng, in zip(dataframe_filtered.location_latitude, dataframe_filtered.location_longitude):
    states.add_child(
        folium.features.Marker(
            [lat, lng],
            popup= dataframe_filtered.index[i],
        )
    )
    i+=1
# add incidents to map
venues_map.add_child(states)
venues_map

In [16]:
bins = pd.IntervalIndex.from_tuples([(0, 100), (101, 200), (201, 300), (301, 400), (401, 500), (501, 600), (601, 700), (701, 800), (801, 900)])
dataframe_filtered['category'] = pd.cut(dataframe_filtered['no_restaurant'], bins)

In [17]:
x = dataframe_filtered.groupby('category').groups.keys()

In [18]:
venues_map = folium.Map(location=[12.934533, 77.626579], zoom_start=11) # generate map centred around Bengaluru

states = folium.map.FeatureGroup()
i=0
for lat, lng, in zip(dataframe_filtered.location_latitude, dataframe_filtered.location_longitude):
    states.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
        )
    )
    i+=1
colors = ['red', 'blue', 'green', 'purple', 'orange', 'white', 'pink', 'gray', 'black']
j=0
for i in range(0,len(list(x))):
    group = dataframe_filtered.groupby('category').get_group(list(x)[i])
    i=0
    for lat, lng, in zip(group.location_latitude, group.location_longitude):
        states.add_child(
            folium.Marker(
                [lat, lng],
                popup= group.index[i],
                icon=folium.Icon(color=colors[j], icon='cloud')
            )
        )
        i+=1
    j+=1
# add incidents to map
venues_map.add_child(states)

# url = ('https://raw.githubusercontent.com/SECOORA/static_assets/'
#        'master/maps/img/rose.png')
# FloatImage(image_file, bottom=40, left=65).add_to(venues_map)
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Number of restaurants</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>0-100</li>
    <li><span style='background:blue;opacity:0.7;'></span>101-200</li>
    <li><span style='background:green;opacity:0.7;'></span>201-300</li>
    <li><span style='background:purple;opacity:0.7;'></span>301-400</li>
    <li><span style='background:orange;opacity:0.7;'></span>401-500</li>
    <li><span style='background:white;opacity:0.7;'></span>501-600</li>
    <li><span style='background:pink;opacity:0.7;'></span>601-700</li>
    <li><span style='background:gray;opacity:0.7;'></span>701-800</li>
    <li><span style='background:black;opacity:0.7;'></span>801-900</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

venues_map.get_root().add_child(macro)

venues_map


In [19]:
### Neighborhood by cost of food
#### Lets see the cost of food in each neighborhood of Bengaluru
rest_df['approx_cost(for two people)'] = rest_df['approx_cost(for two people)'].str.replace(",","").astype(float)

In [20]:
#But before that lets remove all those places that has less than 50 restaurants
above_50 = rest_df.groupby('location')['url'].count()[rest_df.groupby('location')['url'].count() >= 50].index

In [21]:
##### Does this mean higher the cost of food means posche area
rest_df[rest_df['location'].isin(above_50)].groupby('location')['approx_cost(for two people)'].mean().sort_values(ascending= False)

location
Race Course Road                 1309.352518
Lavelle Road                     1307.934990
MG Road                          1155.704698
Infantry Road                    1062.251656
Residency Road                    966.320475
St. Marks Road                    871.306818
Cunningham Road                   864.969450
Church Street                     834.885764
Koramangala 3rd Block             778.472222
Ulsoor                            765.830875
Richmond Road                     756.281095
Sadashiv Nagar                    717.460317
Koramangala 4th Block             696.361849
Seshadripuram                     684.848485
Old Airport Road                  666.162528
Koramangala 5th Block             663.663845
Indiranagar                       653.788027
Brigade Road                      650.935961
Koramangala 2nd Block             622.549020
Koramangala 6th Block             616.302448
Domlur                            605.145228
Whitefield                        598.152836
K

In [22]:
nb_cost_df = rest_df.groupby(['location'])['location_latitude', 'location_longitude'].first()
nb_cost_df['cost'] = rest_df.groupby(['location'])['approx_cost(for two people)'].mean()
nb_cost_df

,location_latitude,location_longitude,cost
location,,,
,12.979120,77.591300,473.804012
BTM,12.911276,77.604565,396.480973
Banashankari,12.915221,77.573598,421.225166
Banaswadi,13.014162,77.651854,397.013783
Bannerghatta Road,12.826493,77.586473,443.679654
Basavanagudi,12.941726,77.575502,360.979532
Basaveshwara Nagar,12.993383,77.538947,429.144385
Bellandur,12.979120,77.591300,527.503888
Bommanahalli,12.908945,77.623904,403.682203


In [23]:
bins = pd.IntervalIndex.from_tuples([(200, 600), (601, 1000), (1001, 1400),])
nb_cost_df['category'] = pd.cut(nb_cost_df['cost'], bins)

In [25]:
x = nb_cost_df.groupby('category').groups.keys()
venues_map = folium.Map(location=[12.934533, 77.626579], zoom_start=11) # generate map centred around Bengaluru

states = folium.map.FeatureGroup()
i=0
for lat, lng, in zip(nb_cost_df.location_latitude, nb_cost_df.location_longitude):
    states.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
        )
    )
    i+=1
colors = ['red', 'blue', 'green', 'purple', 'orange', 'black']
j=0
for i in range(0,len(list(x))):
    group = nb_cost_df.groupby('category').get_group(list(x)[i])
    i=0
    for lat, lng, in zip(group.location_latitude, group.location_longitude):
        states.add_child(
            folium.Marker(
                [lat, lng],
                popup= group.index[i],
                icon=folium.Icon(color=colors[j], icon='cloud')
            )
        )
        i+=1
    j+=1
# add incidents to map
venues_map.add_child(states)

# url = ('https://raw.githubusercontent.com/SECOORA/static_assets/'
#        'master/maps/img/rose.png')
# FloatImage(image_file, bottom=40, left=65).add_to(venues_map)
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Cost</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>200-600</li>
    <li><span style='background:blue;opacity:0.7;'></span>601-1000</li>
    <li><span style='background:green;opacity:0.7;'></span>1001-1400</li>
    <li><span style='background:purple;opacity:0.7;'></span>2201-2600</li>
  </ul>
</div>
</div>
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

venues_map.get_root().add_child(macro)

venues_map
